# TODO: Zklend

In [1]:
from typing import Dict

import IPython.display
import pandas

from classes import State, Prices
import constants
import db

## Load and prepare events

In [ ]:
# Establish the connection.
connection = db.establish_connection()

# Load all Zklend events.
zklend_events = pandas.read_sql(
    sql = 
    f"""
    SELECT
        *
    FROM
        starkscan_events
    WHERE
        from_address='{constants.Protocol.ZKLEND.value}'
    AND
        key_name IN ('Deposit', 'Withdrawal', 'CollateralEnabled', 'CollateralDisabled', 'Borrowing', 'Repayment', 'Liquidation', 'AccumulatorsSync')
    ORDER BY
        block_number ASC, id;
    """,
    con = connection,
)

# Load all zTokens.
ztoken_events = pandas.read_sql(
    sql = 
    f"""
    SELECT
        *
    FROM
        starkscan_events
    WHERE
        from_address IN (
            '0x01b5bd713e72fdc5d63ffd83762f81297f6175a5e0a4771cdadbc1dd5fe72cb1',
            '0x047ad51726d891f972e74e4ad858a261b43869f7126ce7436ee0b2529a98f486',
            '0x00811d8da5dc8a2206ea7fd0b28627c2d77280a515126e62baa4d78e22714c4a',
            '0x062fa7afe1ca2992f8d8015385a279f49fad36299754fb1e9866f4f052289376',
            '0x02b9ea3acdb23da566cee8e8beae3125a1458e720dea68c4a9a7a2d8eb5bbb4a'
        ) AND key_name = 'Transfer'
    ORDER BY
        block_number ASC, id;
    """,
    con = connection,
)

# Close the connection.
connection.close()

In [10]:
zklend_events.set_index('id', inplace = True)

# Process events

Events examples:
- [Deposit](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_3)
- [AccumulatorsSync](https://starkscan.co/event/0x029628b89875a98c1c64ae206e7eb65669cb478a24449f3485f5e98aba6204dc_0)
- [CollateralEnabled](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_6)
- [CollateralDisabled](https://starkscan.co/event/0x0049b445bed84e0118795dbd22d76610ccac2ad626f8f04a1fc7e38113c2afe7_0)
- [Withdrawal](https://starkscan.co/event/0x03472cf7511687a55bc7247f8765c4bbd2c18b70e09b2a10a77c61f567bfd2cb_4)
- [Borrowing](https://starkscan.co/event/0x076b1615750528635cf0b63ca80986b185acbd20fa37f0f2b5368a4f743931f8_3)
- [Repayment](https://starkscan.co/event/0x06fa3dd6e12c9a66aeacd2eefa5a2ff2915dd1bb4207596de29bd0e8cdeeae66_5)
- [Liquidation](https://starkscan.co/event/0x07b8ec709df1066d9334d56b426c45440ca1f1bb841285a5d7b33f9d1008f256_5)

In [11]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
prices = Prices()
for _, event in zklend_events.iterrows():
    state.process_event(event = event)

# TODO: Debugging negative deposits below

In [16]:
# TODO: negative deposits?
print("Action   Token   Raw value   Raw $   Readable $")
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min(), f"${prices.to_dollars(pandas.Series(values).min(), token)}", prices.to_dollars_pretty(pandas.Series(values).min(), token))

Action   Token   Raw value   Raw $   Readable $
deposit ETH -300437786.967166206714 $-5.576395720118875000275964687E-7 $0
deposit wBTC 0 $0 $0
deposit USDC -1.39246404407114132779 $-0.000001393856508115212315759982245 $0
deposit DAI 0 $0E-53 $0
deposit USDT -4997676.927172805096183238170 $-4.995722835494280378399430221 $-4.99572
borrowings ETH 0 $0E-42 $0
borrowings wBTC 0 $0 $0
borrowings USDC 0 $0E-51 $0
borrowings DAI 0 $0E-53 $0
borrowings USDT 0 $0E-51 $0


In [13]:
# TODO: show negative deposits
[
    (user, user_state.token_states['USDT'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['USDT'].deposit < 0
]

[('0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1',
  Decimal('-4997676.927172805096183238170'))]

In [14]:
# TODO: Get token prices here: https://www.coingecko.com/en/api/documentation